In [2]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.neighbors import NearestNeighbors
from sklearn.externals import joblib
from tqdm import tqdm

df_features = pd.read_excel('Hot 100 Audio Features.xlsx')
df_features

,SongID,Performer,Song,spotify_genre,spotify_track_id,spotify_track_preview_url,spotify_track_album,spotify_track_explicit,spotify_track_duration_ms,spotify_track_popularity,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,"AdictoTainy, Anuel AA & Ozuna","Tainy, Anuel AA & Ozuna",Adicto,['pop reggaeton'],3jbT1Y5MoPwEIpZndDDwVq,NaN,Adicto (with Anuel AA & Ozuna),0.0,270740.0,91.0,...,10.0,-4.803,0.0,0.0735,0.017000,0.000016,0.1790,0.623,80.002,4.0
1,The Ones That Didn't Make It Back HomeJustin M...,Justin Moore,The Ones That Didn't Make It Back Home,"['arkansas country', 'contemporary country', '...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ShallowLady Gaga & Bradley Cooper,Lady Gaga & Bradley Cooper,Shallow,"['dance pop', 'pop']",2VxeLyX666F8uXCJ0dZF8B,NaN,A Star Is Born Soundtrack,0.0,215733.0,88.0,...,7.0,-6.362,1.0,0.0308,0.371000,0.000000,0.2310,0.323,95.799,4.0
3,EnemiesPost Malone Featuring DaBaby,Post Malone Featuring DaBaby,Enemies,"['dfw rap', 'melodic rap', 'rap']",0Xek5rqai2jcOWCYWJfVCF,NaN,Hollywood's Bleeding,1.0,196760.0,86.0,...,6.0,-4.169,1.0,0.2100,0.058800,0.000000,0.0955,0.667,76.388,4.0
4,"Bacc At It AgainYella Beezy, Gucci Mane & Quavo","Yella Beezy, Gucci Mane & Quavo",Bacc At It Again,"['dfw rap', 'rap', 'southern hip hop', 'trap']",2biNa12dMbHJrHVFRt8JyO,https://p.scdn.co/mp3-preview/fa6fa6f6f363be29...,Bacc At It Again,1.0,228185.0,61.0,...,8.0,-5.725,0.0,0.1680,0.001240,0.000001,0.0716,0.856,135.979,4.0
5,The ArcherTaylor Swift,Taylor Swift,The Archer,"['dance pop', 'pop', 'post-teen pop']",3pHkh7d0lzM2AldUtz2x37,NaN,Lover,0.0,211240.0,76.0,...,0.0,-9.375,1.0,0.0401,0.120000,0.005690,0.0663,0.166,124.344,4.0
6,VIBEZDaBaby,DaBaby,VIBEZ,"['nc hip hop', 'rap']",0fySG6A6qLE8IvDpayb5bM,NaN,KIRK,1.0,144935.0,88.0,...,1.0,-2.708,0.0,0.3070,0.113000,0.000000,0.1070,0.777,154.187,4.0
7,When I Grow UpNF,NF,When I Grow Up,"['hip hop', 'pop rap', 'rap']",0hWzB4dR1zwcokPvccww0k,NaN,When I Grow Up,0.0,196800.0,74.0,...,2.0,-3.985,1.0,0.1850,0.320000,0.000000,0.1520,0.247,109.994,4.0
8,Hollywood's BleedingPost Malone,Post Malone,Hollywood's Bleeding,"['dfw rap', 'melodic rap', 'rap']",7sWRlDoTDX8geTR8zzr2vt,NaN,Hollywood's Bleeding,0.0,156266.0,85.0,...,4.0,-3.206,0.0,0.0476,0.331000,0.000000,0.1030,0.170,130.218,4.0
9,What Happens In A Small TownBrantley Gilbert +...,Brantley Gilbert + Lindsay Ell,What Happens In A Small Town,NaN,44UTzjhEM825Sw0qC89oGK,NaN,Fire & Brimstone,0.0,203560.0,57.0,...,11.0,-5.361,1.0,0.0402,0.000934,0.000000,0.1300,0.457,138.171,4.0


In [4]:
df1 = df_features
df1 = df1.drop(columns=['instrumentalness','spotify_track_preview_url','spotify_track_id'])
df1 = df1.dropna()
df1 = df1.reset_index()
df1

,index,SongID,Performer,Song,spotify_genre,spotify_track_album,spotify_track_explicit,spotify_track_duration_ms,spotify_track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,liveness,valence,tempo,time_signature
0,0,"AdictoTainy, Anuel AA & Ozuna","Tainy, Anuel AA & Ozuna",Adicto,['pop reggaeton'],Adicto (with Anuel AA & Ozuna),0.0,270740.0,91.0,0.734,0.836,10.0,-4.803,0.0,0.0735,0.017000,0.1790,0.623,80.002,4.0
1,2,ShallowLady Gaga & Bradley Cooper,Lady Gaga & Bradley Cooper,Shallow,"['dance pop', 'pop']",A Star Is Born Soundtrack,0.0,215733.0,88.0,0.572,0.385,7.0,-6.362,1.0,0.0308,0.371000,0.2310,0.323,95.799,4.0
2,3,EnemiesPost Malone Featuring DaBaby,Post Malone Featuring DaBaby,Enemies,"['dfw rap', 'melodic rap', 'rap']",Hollywood's Bleeding,1.0,196760.0,86.0,0.542,0.674,6.0,-4.169,1.0,0.2100,0.058800,0.0955,0.667,76.388,4.0
3,4,"Bacc At It AgainYella Beezy, Gucci Mane & Quavo","Yella Beezy, Gucci Mane & Quavo",Bacc At It Again,"['dfw rap', 'rap', 'southern hip hop', 'trap']",Bacc At It Again,1.0,228185.0,61.0,0.948,0.623,8.0,-5.725,0.0,0.1680,0.001240,0.0716,0.856,135.979,4.0
4,5,The ArcherTaylor Swift,Taylor Swift,The Archer,"['dance pop', 'pop', 'post-teen pop']",Lover,0.0,211240.0,76.0,0.292,0.574,0.0,-9.375,1.0,0.0401,0.120000,0.0663,0.166,124.344,4.0
5,6,VIBEZDaBaby,DaBaby,VIBEZ,"['nc hip hop', 'rap']",KIRK,1.0,144935.0,88.0,0.768,0.652,1.0,-2.708,0.0,0.3070,0.113000,0.1070,0.777,154.187,4.0
6,7,When I Grow UpNF,NF,When I Grow Up,"['hip hop', 'pop rap', 'rap']",When I Grow Up,0.0,196800.0,74.0,0.817,0.814,2.0,-3.985,1.0,0.1850,0.320000,0.1520,0.247,109.994,4.0
7,8,Hollywood's BleedingPost Malone,Post Malone,Hollywood's Bleeding,"['dfw rap', 'melodic rap', 'rap']",Hollywood's Bleeding,0.0,156266.0,85.0,0.405,0.646,4.0,-3.206,0.0,0.0476,0.331000,0.1030,0.170,130.218,4.0
8,10,Famous HoesNLE Choppa,NLE Choppa,Famous Hoes,['memphis hip hop'],Famous Hoes,1.0,135460.0,80.0,0.732,0.627,4.0,-5.693,0.0,0.1610,0.346000,0.2610,0.332,81.481,4.0
9,11,Cornelia StreetTaylor Swift,Taylor Swift,Cornelia Street,"['dance pop', 'pop', 'post-teen pop']",Lover,0.0,287266.0,74.0,0.824,0.624,0.0,-9.728,1.0,0.0827,0.781000,0.1000,0.248,102.012,4.0


In [5]:


a=0
dict = {}
dfObj = pd.DataFrame(columns=['genre'])
for i in tqdm(range(df1.shape[0])):
    s = df1.loc[i,'spotify_genre']
    s = s[1:len(s)-1]
    try:
        s=s.split(', ')
        for j in range(len(s)):
            s[j]= s[j][1:len(s[j])-1]
#         print(s[j]) 
    except:
        a +=1 
        pass
    dfObj = dfObj.append({'genre':s}, ignore_index=True)
    df1.loc[i,'spotify_genre'] = s[0]
#     df1.loc[i,'spotify_genre'] = s
#     print(s)

#     df1.loc[1,'spotify_genre']
# range(df.shape[0])
print(a)
#  df1.loc[1,'spotify_genre']
# df1['spotify_genre']

100%|███████████████████████████████████████████████████████████████████████████| 23559/23559 [00:50<00:00, 463.18it/s]


0


In [300]:
# for i in tqdm(range(df1.shape[0])):
dfObj['genre'].values[1][1]
len(df1.spotify_genre.unique())
df1

,index,Performer,Song,spotify_genre,spotify_track_album,spotify_track_explicit,spotify_track_duration_ms,spotify_track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,liveness,valence,tempo,time_signature
0,0,"Tainy, Anuel AA & Ozuna",Adicto,pop reggaeton,Adicto (with Anuel AA & Ozuna),0.0,270740.0,91.0,0.734,0.836,10.0,-4.803,0.0,0.0735,0.017000,0.1790,0.623,80.002,4.0
1,2,Lady Gaga & Bradley Cooper,Shallow,dance pop,A Star Is Born Soundtrack,0.0,215733.0,88.0,0.572,0.385,7.0,-6.362,1.0,0.0308,0.371000,0.2310,0.323,95.799,4.0
2,3,Post Malone Featuring DaBaby,Enemies,dfw rap,Hollywood's Bleeding,1.0,196760.0,86.0,0.542,0.674,6.0,-4.169,1.0,0.2100,0.058800,0.0955,0.667,76.388,4.0
3,4,"Yella Beezy, Gucci Mane & Quavo",Bacc At It Again,dfw rap,Bacc At It Again,1.0,228185.0,61.0,0.948,0.623,8.0,-5.725,0.0,0.1680,0.001240,0.0716,0.856,135.979,4.0
4,5,Taylor Swift,The Archer,dance pop,Lover,0.0,211240.0,76.0,0.292,0.574,0.0,-9.375,1.0,0.0401,0.120000,0.0663,0.166,124.344,4.0
5,6,DaBaby,VIBEZ,nc hip hop,KIRK,1.0,144935.0,88.0,0.768,0.652,1.0,-2.708,0.0,0.3070,0.113000,0.1070,0.777,154.187,4.0
6,7,NF,When I Grow Up,hip hop,When I Grow Up,0.0,196800.0,74.0,0.817,0.814,2.0,-3.985,1.0,0.1850,0.320000,0.1520,0.247,109.994,4.0
7,8,Post Malone,Hollywood's Bleeding,dfw rap,Hollywood's Bleeding,0.0,156266.0,85.0,0.405,0.646,4.0,-3.206,0.0,0.0476,0.331000,0.1030,0.170,130.218,4.0
8,10,NLE Choppa,Famous Hoes,memphis hip hop,Famous Hoes,1.0,135460.0,80.0,0.732,0.627,4.0,-5.693,0.0,0.1610,0.346000,0.2610,0.332,81.481,4.0
9,11,Taylor Swift,Cornelia Street,dance pop,Lover,0.0,287266.0,74.0,0.824,0.624,0.0,-9.728,1.0,0.0827,0.781000,0.1000,0.248,102.012,4.0


In [7]:

df_artist = pd.DataFrame(columns=['artist'])

for i in tqdm(range(df1.shape[0])):
    s = df1.loc[i,'Performer']
    s = s.replace(' &',',')
    s = s.replace(' Featuring',',')
    s = s.rsplit(', ')
    df_artist = df_artist.append({'artist':s}, ignore_index=True)
    df1.loc[i,'Performer'] = s[0]    

100%|███████████████████████████████████████████████████████████████████████████| 23559/23559 [00:44<00:00, 523.96it/s]


In [9]:
dfObj['artist'] = df_artist['artist']
df1.drop(columns='index','')
df1.to_csv('Hot 100 Audio Features(Cleaned).csv')
dfObj.to_csv('Feature arrays.csv')